# <center>Machine learning from scratch - Part III</center>
## <center>WebValley ReImagined 2020</center>
### <center>Marco Chierici</center>
#### <center>FBK/MPBA</center>

Recap. We are using a subset of the SEQC neuroblastoma data set [Zhang et al, Genome Biology, 2015] consisting of 272 samples (136 training, 136 test). The data was preprocessed a bit to facilitate the progress of the tutorial.

We start by loading the modules we need to process the data.

In [ ]:
import numpy as np
import pylab as plt ## for plotting
import pandas as pd ## for reading text files and manipulating data frames
from sklearn import metrics
from sklearn import neighbors ## kNN classifier
from pathlib import Path ## for creating paths in a neat way
%matplotlib inline
np.random.seed(42) ## set random seed just in case

# 1. Data loading

Let's start from scratch by reloading the Neuroblastoma data, preparing them for a classification task on the "CLASS" label (see part2 notebook):

In [ ]:
##  for convenience, define the data directory as a variable
DATA_DIR = Path("data")
DATA = DATA_DIR / "MAV-G_272.txt"
LABS = DATA_DIR / "labels.txt"
data = pd.read_csv(DATA, sep = "\t")

# We drop the first column from the train and test expression sets, since it's just the sample IDs...
data = data.drop('sampleID', axis=1)
# ...and store the data into Numpy arrays.
X = data.values
# Now we read in the files containing labels and select the column with the CLASS target
labs = pd.read_csv(LABS, sep = "\t")
class_lab = labs[['CLASS']]
y = class_lab.values.ravel()

In [ ]:
# following our consolidated practice, print the shape of the data as a sanity check:
X.shape

In [ ]:
y.shape

# 2. Data partitioning

### Hold-out strategy

The idea behind data partitioning is to split your original data set into a **train** portion (for developing your machine learning model) and a **test** portion (for evaluating the performance of the trained model).

The simplest and most straightforward way to partition your data set is to randomly split it in two groups (*hold-out strategy*).

You achieve this using scikit-learn's function `train_test_split`, in the `model_selection` submodule.

For example, let's split the data (X) into 80% train and 20% test (note the argument `test_size=0.2`). Use a random_state of your choice:

In [ ]:
from sklearn.model_selection import train_test_split
x_tr, x_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.2, random_state=78)

# 3. Data preprocessing

In [ ]:
from sklearn import preprocessing
## first you need to create a "scaler" object
scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
## then you actually scale data by fitting the scaler object on the data
scaler.fit(x_tr)
x_tr = scaler.transform(x_tr)
x_ts = scaler.transform(x_ts)

# 4. Supervised Learning

Scikit-learn provides you access to several models via a very convenient _fit_ and _predict_ interface.

## 4.1 k-NN classifier

Let's fit again a k-NN model on the whole training data and then use it to predict the labels of the test data. This time we'll use a different number of neighbors, k:

In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors=5)

In [ ]:
knn.fit(x_tr, y_tr)

In [ ]:
y_pred_knn = knn.predict(x_ts) # predict labels on test data

_In general, a classifier has **parameters** that need to be tuned. Default choices are not good in all situations._

_For example, in k-NN the main parameter is the **number of neighbors** used in the nearest neighbors algorithm._

## Performance assessment: Confusion matrix

Confusion matrix recap:

|      |  |  Predicted  |    |
|------|-----------|----|----|
|      |           | 0 | 1  |
| True | 0        | TN | FP |
|      | 1         | FN | TP |


In [ ]:
conf = metrics.confusion_matrix(y_ts, y_pred_knn)
conf

The total number of class 0 test samples (AN = All Negatives) should be equal to the sum of the first row of the confusion matrix, i.e., TN + FP:

In [ ]:
np.sum(y_ts == 0) # total number of "class 0" samples in the test set

Similarly for class 1, i.e., AP = All Positives = TP + FN:

In [ ]:
np.sum(y_ts==1) # total number of "class 1" samples in the test set

To compute the metrics, we'll take the scikit-learn shortcut instead of recomputing them by hand. I'm also throwing in a couple more metrics: precision and F1-score.

In [ ]:
print(f"MCC = {metrics.matthews_corrcoef(y_ts, y_pred_knn):.3f}")
print(f"Accuracy = {metrics.accuracy_score(y_ts, y_pred_knn):.3f}")
print(f"Sensitivity = {metrics.recall_score(y_ts, y_pred_knn):.3f}")
print(f"Precision = {metrics.precision_score(y_ts, y_pred_knn):.3f}")
print(f"F1-score = {metrics.f1_score(y_ts, y_pred_knn):.3f}")

So far we focused on the k-NN classifiers. However, we already explored theoretical aspects related to two other broadly used classifiers: Support Vector Machines (SVMs) and Random Forests (RFs). In this part of tutorial, the first thing we want to do is assessing how these two alternative classification methods perform on our neuroblastoma dataset.

## 4.2 Support Vector Machines

We start with SVM. We first rescale the data, import the relevant model and create an instance of the SVM classifier. Use the same seed used previously.

In [ ]:
# reminder: x_tr, x_ts were previously rescaled using the MinMaxScaler.
# Now we want to standardize the features, so we need to recreate the original arrays:
x_tr, x_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.2, random_state=78)

scaler = preprocessing.StandardScaler()
## then you actually scale data by fitting the scaler object on the data
scaler.fit(x_tr)
x_tr = scaler.transform(x_tr)
x_ts = scaler.transform(x_ts)

In [ ]:
## import support vector classifier (SVC) and create an instance
from sklearn.svm import SVC
svc = SVC(kernel='linear')

Note that the specification _kernel = 'linear'_ implies that a linear kernel will be used. If you remember from the lecture, this means that a linear function is used to define the decision boundaries of the classifier. Alternatives include _‘poly’_ and _‘rbf’_ for polynomial or gaussian kernels respectively. Scikit-learn offers an alternative implementation of linear SVMs. You can find more details in Scikit User Guide. 

**Tip:** always start experimenting with the linear kernel, which is the simplest one; try more complex kernels later on (Occam's razor...)

As previously done with the k-NN classifier, we fit an SVM model on the training data:

In [ ]:
svc.fit(x_tr, y_tr)

Simple as that. Now we have a fitted SVM model that we can use to make predictions on new data.

Of course there are a few parameters that may require tuning: more on this later.

### Making predictions

In [ ]:
y_pred_svm = svc.predict(x_ts)

We now have a set of predictions for each entry in the test set. Since we also have the actual labels for each record in the test set, we can use them to assess the performance of the SVM model.

Now we give a look at the classification metrics introduced in the first part of the tutorial.

In [ ]:
print(f"MCC = {metrics.matthews_corrcoef(y_ts, y_pred_svm):.3f}")
print(f"ACC = {metrics.accuracy_score(y_ts, y_pred_svm):.3f}")
print(f"SENS = {metrics.recall_score(y_ts, y_pred_svm):.3f}")

## 4.3 Random Forest

Thanks to scikit-learn simple API, we can easily create a RandomForest instance and fit it to our training data. Note that the Random Forest is robust to feature rescaling (and also supports categorical features), so it is not necessary to apply `MinMaxScaler` or `StandardScaler`.

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC
# x_tr, x_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.2, random_state=78)

clf = RFC(n_estimators=500, random_state=42)
clf.fit(x_tr, y_tr)

### Making predictions

In [ ]:
y_pred_rfc = clf.predict(x_ts)

Compute a few metrics using the actual test set labels:

In [ ]:
print(f"MCC = {metrics.matthews_corrcoef(y_ts, y_pred_rfc):.3f}")
print(f"ACC = {metrics.accuracy_score(y_ts, y_pred_rfc):.3f}")
print(f"SENS = {metrics.recall_score(y_ts, y_pred_rfc):.3f}")

How did you do? Note that (not surprisingly) there is an element of randomness in a "random forest". 

I'm getting an accuracy of about 94-98%, but "your mileage may vary": the precise number will be different each time you run the algorithm.

### Probabilities

Depending on the algorithm, some of the supervised models in scikit-learn can also provide the **probability** that a record belongs in each category. 

For random forest, we can obtain these probabilities by calling the `predict_proba` method on the fitted model. Because we have two categories, 0 (negative) and 1 (positive), scikit-learn will return two category probabilities (the sum across all categories will add up to 1).

Looking at the prediction probabilities may be useful to understand on which samples the classifier is "unsure" (i.e., the probabilities are around 0.5 for both classes).

In [ ]:
prob_rfc = clf.predict_proba(x_ts)

In [ ]:
print(prob_rfc)

## Trying different classifiers

scikit-learn makes it easy to try different classifiers. 

Neural networks, naive bayes, random forest, logistic regression, support vector machines, and other algorithms all have a very similar interface in sklearn (of course the underlying mathematics can be dramatically different!) 

1. you create an instance of the model (in the variable, say, `clf`), optionally tuning the parameters; 
2. you fit the model on some training data (`clf.fit(x_tr, y_tr)`); 
3. you predict the labels of unseen test data (`clf.predict(x_ts)`);
4. you evaluate the performance.


### Exercise: neural net!

To use a neural net, you only need to make a few changes to this workbook.

First, load the appropriate library

from sklearn.neural_network import MLPClassifier

Then, call a neural network classifier rather than a random forest model

clf = MLPClassifier()

Proceed as usual to fit and make predictions.

_Et voilà!_ In seconds you are making classification predictions using a neural net model rather than a random forest!

* Train a neural net and assess its performance on the test set;
* Compare the performance of the classifiers you just used;
* Given the performance metrics you assessed, can you say there is a "best" classification algorithm? Do you think this algorithm will perform better on a different task / data?

Hint: the effectiveness depends on a number of factors, mainly the **classifier parameters** and the **data**. Some data sets just match very well to a particular prediction algorithm.


* Compare the metrics of the different classifiers. What can you say about this classification task? Do the classifiers learn something?
* Which classifier has the best accuracy and MCC?
* Are you confident enough that such classifier is able to *generalize* beyond its training set?
* Do you know if there is a more robust way to assess the performance of the models?


# 5. Cross-validation

So far we fitted several models on our training data, then we predicted the labels on the test data, computing a set of metrics.

How can we know if our model is going to generalize well on new unseen data?

We need a more robust way to _estimate_ the model performance and its generalization capabilities.

We already used the **hold-out strategy**, with scikit's `train_test_split` function, to split our `X` data into one training/test partition.

Partitioning the dataset once is not enough. The partitions depend on the random seed used in the splitting function.

More robust strategies involve splitting the data in **multiple (complementary) subsets**.

One of such strategies is the **k-fold cross-validation**, introduced during the lecture:

![k-fold cv](https://www.researchgate.net/profile/B_Aksasse/publication/326866871/figure/fig2/AS:669601385947145@1536656819574/K-fold-cross-validation-In-addition-we-outline-an-overview-of-the-different-metrics-used_W640.jpg)

Example of a 5-fold cross-validation (CV) with scikit-learn:

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

A "stratified" 5-fold CV means that the folds are made by preserving the percentage of samples for each class.

_Recap:_ the same random_state will generate the same splits. This is useful for reproducibility.

To actually get the splitting indices and create the folds, we need to iterate over the `skf` object. Note that here I am using a Random Forest: feel free to experiment with other classifiers!

In [ ]:
## get the number of splitting operations
N = skf.get_n_splits(X, y)

## reinitialize a classifier
clf = RFC(n_estimators=500)

## create empty lists to store the CV metrics
acc_list = []
mcc_list = []

## split data and iterate over the splits,
## computing classifier accuracy & MCC on each test partition
n_fold = 1
for (idx_tr, idx_ts) in skf.split(X, y):
    print("### Fold ", n_fold)
    X_train, Y_train = X[idx_tr], y[idx_tr]
    X_test, Y_test = X[idx_ts], y[idx_ts]
    print()
    clf.fit(X_train, Y_train)
    Y_test_pred = clf.predict(X_test)
    acc = metrics.accuracy_score(Y_test, Y_test_pred)
    mcc = metrics.matthews_corrcoef(Y_test, Y_test_pred)
    print(f"Accuracy on TEST set: {acc:.3f}")
    print(f"MCC on TEST set: {mcc:.3f}")
    print()
    ## append values to lists
    acc_list.append(acc)
    mcc_list.append(mcc)
    
    n_fold = n_fold + 1

*Q: how are the computed metrics on the different folds?*

To get an estimate of the predictive performance of our model, we can average over the cross-validation metrics.

In [ ]:
## note: we need to convert the lists to numpy arrays before computing the means
acc_avg = np.mean(np.array(acc_list))
mcc_avg = np.mean(np.array(mcc_list))

print(f"Average cross-validation accuracy: {acc_avg:.3f}")
print(f"Average cross-validation MCC: {mcc_avg:.3f}")

### Remarks

If you want to be cool and have an even better estimate of how the model can generalize on new data, you can **repeat the cross-validation several times** ("iterated cross-validation"). Each time you need using a different random_state for generating the splits.

Moreover, the average alone is not sufficient as you should also capture the **dispersion** of the values around the average. This can be done, for example, by assessing the standard deviation or, even better, by computing **confidence intervals**.

# 6. Feature ranking

One type of insight you can gain from a machine learning model is the feature importance (or weight). Which genes are most likely to influence the classification of our neuroblastoma patients? Are some features pivotal, and others largely ignored?

Because a Random Forest model branches repeatedly on different features, the model becomes "aware" of which features are particularly influential in classifiying a patient. 

Scikit-learn allows us to read this information off of a trained Random Forest model through the `feature_importances_` attribute (mind the trailing underscore!).

In [ ]:
# Retrain a random forest
rf = RFC(n_estimators=500)
rf.fit(x_tr, y_tr)

For the sake of completeness make the predictions and check the classification performance.

In [ ]:
y_pred_rfc = rf.predict(x_ts)
print(f"MCC = {metrics.matthews_corrcoef(y_ts, y_pred_rfc):.3f}")
print(f"ACC = {metrics.accuracy_score(y_ts, y_pred_rfc):.3f}")
print(f"SENS = {metrics.recall_score(y_ts, y_pred_rfc):.3f}")

Now extract the feature importances and display the first 10:

In [ ]:
# get the importances
importances = rf.feature_importances_
# sort by decreasing importance
indices = np.argsort(importances)[::-1]
# get the gene names
genes = data.columns.values
# print the feature ranking
print("Feature ranking (top 10 features):")
for f in range(10):
    print(genes[indices[f]], " - ", importances[indices[f]])

A **stem plot** is a common way to visually represent this kind of information:

In [ ]:
n_feat = 10
plt.figure()
plt.title("Feature importances")
plt.stem(range(n_feat), importances[indices[:n_feat]])
plt.xticks(range(n_feat), genes[indices[:n_feat]], rotation="vertical")
plt.xlim([-1, n_feat])
plt.show()

# 7. Further topics

## Parameter tuning

As mentioned in the lecture, Scikit learn offers a very useful and flexible tool for parameter tuning called _GridSearchCV_. While the tool is very sophisticated and efficient, it is useful to at least try an example _by hand_ to understand what is happening in the background.

For this example we use a linear SVM and try to tune the C parameter. You might remember from the lectures that the paramenter C essentially controls how much we want to avoid misclassifying each training example. Large values of C result in smaller margins, i.e. closer fitting to the training data. As mentioned in the classes, the drawback is over-fitting, resulting in poor generalization.

In [ ]:
# first of all, let's get a clean train/test split of the original data
x_tr, x_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.2, random_state=78)
# rescale
scaler = preprocessing.StandardScaler()
scaler.fit(x_tr)
x_tr = scaler.transform(x_tr)
x_ts = scaler.transform(x_ts)

In [ ]:
## define the sequence of C values we want to use in the search of the best one
C_list = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1]
for C in C_list:
    print(f'C = {C}')
    svc = SVC(kernel='linear', C=C)
    svc.fit(x_tr, y_tr)
    class_pred_ts = svc.predict(x_ts)
    print(f'MCC = {metrics.matthews_corrcoef(y_ts, class_pred_ts):.3f}')
    print(f'ACC = {metrics.accuracy_score(y_ts, class_pred_ts):.3f}')
    print(f'SENS = {metrics.recall_score(y_ts, class_pred_ts):.3f}', '\n')

Depending on the splits I generated with train_test_split, I get the highest MCC for C=1e-5, which I take as the optimal parameter in this setting. You may get different values.

**Optional exercise:** as you already saw in the lectures, there are many parameters that can be tuned, also when considering only one simple classifier. For example, if you consider SVM with 'rbf' kernel, you could check performance changes with different values of C **and** gamma, for example using two nested loops.

In [ ]:
## space for exercise

As we mentioned, Scikit offers fully automated parameter tuning engine. We illustrate its power with a simple example on our data. We use GridSearchCV to search through a grid of C and gamma parameter options for SVM with 'rbf' kernel.

In [ ]:
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit

C_range = [0.001, 0.01, 0.1, 1, 10, 100]
gamma_range = ['auto', 0.01, 1]
# the parameter grid is defined as a dictionary {<parameter>: <values>} for each parameter
param_grid = dict(gamma=gamma_range, C=C_range)
# define the type of cross-validation for the grid search
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.5, random_state=42)
# create a GridSearchCV object
grid = GridSearchCV(SVC(kernel="rbf"), param_grid=param_grid, cv=cv, n_jobs=4)
# go!
grid.fit(x_tr, y_tr)

The best parameters and the corresponding average cross-validated score are available in the `best_params_` and `best_score_` attributes, respectively:

In [ ]:
print(grid.best_params_)

In [ ]:
print(grid.best_score_)

_Note: GridSearchCV by default does not maximize the MCC, but the accuracy. It is however possible to specify a custom metric to be maximized (or minimized) using sklearn's make_scorer() function._

In [ ]:
# from sklearn.metrics import make_scorer
# mcc_scorer = make_scorer(metrics.matthews_corrcoef)
# grid = GridSearchCV(SVC(kernel="rbf"), param_grid=param_grid, cv=cv, n_jobs=4, scorer=mcc_scorer)

Train a model with the best parameters and predict on the test set, computing a few metrics:

In [ ]:
clf = SVC(kernel="rbf", C=grid.best_params_['C'], gamma=grid.best_params_['gamma'])
clf.fit(x_tr, y_tr)
y_pred = clf.predict(x_ts)
print(f'MCC = {metrics.matthews_corrcoef(y_ts, y_pred):.3f}')
print(f'ACC = {metrics.accuracy_score(y_ts, y_pred):.3f}')
print(f'SENS = {metrics.recall_score(y_ts, y_pred):.3f}')

## Implementing a basic Data Analysis Protocol

As a final example, we implement a 10x 5-fold Cross-validation schema with a simple feature ranking. 

For each CV iteration, a Random Forest model is trained on the training portion of the data, then features are ranked according to the Random Forest importances; a series of Random Forest models are built upon an increasing number of the ranked features (i.e., 1, 5, 10, etc.) and evaluated on the test data in terms of MCC.

The average MCC over the 10x5 CV iterations is computed for the different feature set sizes. We choose the feature set size that maximizes the average MCC.

This basic example is meant as a starting point for building more complex pipelines, i.e., with more feature steps, confidence intervals for MCC, computation of a unified ranked feature list (as in Jurman et al., _Bioinformatics_ , 2008).

In [ ]:
CV_N = 10 # number of CV iterations
CV_K = 5 # number of CV folds
FEATURE_STEPS = [1, 5, 10, 25, 50, 100]
# prepare output MCC array
MCC = np.empty((CV_K*CV_N, len(FEATURE_STEPS)))

In [ ]:
for n in range(CV_N):
    print("~~~ Iteration %d ~~~" % (n+1))
    skf = StratifiedKFold(n_splits=CV_K, shuffle=True, random_state=n)
    for i, (idx_tr, idx_ts) in enumerate(skf.split(x_tr, y_tr)):
        print("Fold %d" % (i+1))
        X_train, Y_train = x_tr[idx_tr], y_tr[idx_tr]
        X_test, Y_test = x_tr[idx_ts], y_tr[idx_ts]
        
        clf = RFC(n_estimators=500, random_state=n)
        clf.fit(X_train, Y_train)
        ranking = np.argsort( clf.feature_importances_ )[::-1]
        
        for j, s in enumerate(FEATURE_STEPS):
            v = ranking[:s] # consider the top s ranked features
            X_tr_fs, X_ts_fs = X_train[:, v], X_test[:, v] # extract them from internal train and test data
            clf.fit(X_tr_fs, Y_train) # train a classifier on the reduced train dataset
            yp = clf.predict(X_ts_fs) # predict on the reduced test dataset
            MCC[(n*CV_K)+i, j] = metrics.matthews_corrcoef(Y_test, yp) # evaluate the model performance
        
    print()


In [ ]:
# np.save("MCC_CV", MCC)
# MCC = np.load("MCC_CV.npy")
MCC.shape

In [ ]:
MCC_avg = np.mean(MCC, axis=0)
MCC_max = np.max(MCC_avg)
n_feats = FEATURE_STEPS[np.argmax(MCC_avg)]

In [ ]:
# average MCC for each feature step
for nf, mcc in zip(FEATURE_STEPS, MCC_avg):
    print("nf = %d, MCC = %.2f" % (nf, mcc))

print()

print("Best MCC = %.2f with %d features" % (MCC_max, n_feats))

In [ ]:
plt.figure()
plt.title("Average MCC")
plt.plot(FEATURE_STEPS, MCC_avg, 'o-')
plt.ylim((0.0, 1.0))
plt.xlabel("Feature steps")
plt.ylabel("MCC")
plt.show()